# This is why South Korea is the only country that minimized the Corona pandemic. It has used data-driven methods and did not force curfews on the residents. How can other countries profit from it before the second wave of infected will hit them?

#### Authors: SmarteDaten & friendlypotato
#### Date: 01.04.2020
<br>

## Content
### 1. Data sources, imports & manipulations
### 2. Initial situation in South Korea
### 3. Exploratory data analysis
### 4. Discussion & recommendations
<br>

In [ ]:
from IPython.display import Image
Image("../input/coronavirus-images-data/coronavirus.jpg")

###### source: https://www.kbv.de/images/contentbilder_m/teaser_corona.png
<br>
<br>

## 1. Data sources, imports & manipulations

In [ ]:
#import dependencies

from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime
%matplotlib inline

from plotly.offline import iplot
from plotly import tools
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

from bokeh.plotting import output_notebook, figure, show
from bokeh.models import ColumnDataSource, Div, Select, Button, ColorBar, CustomJS
from bokeh.layouts import row, column, layout
from bokeh.transform import cumsum, linear_cmap
from bokeh.palettes import Blues8, Spectral3
from bokeh.plotting import figure, output_file, show

import folium 
from folium import plugins
#plt.style.use("fivethirtyeight")# for pretty graphs

import os
#os.chdir("/data/shared/ch00tnb/Corona_datavis")

from datetime import datetime, timedelta

plt.style.use('dark_background')


#Data Sources
#Coronavirus: https://ourworldindata.org/coronavirus-source-data
#Geocode by country: https://developers.google.com/public-data/docs/canonical/countries_csv
#Population by country: https://data.worldbank.org/indicator/sp.pop.totl

#read in current data
full_data = pd.read_csv("https://covid.ourworldindata.org/data/ecdc/full_data.csv")

#read in population data
population_data = pd.read_csv("../input/coronavirus-images-data/API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv", skiprows=3)

#overwrite old with current data
#full_data.to_csv("../input/coronavirus-images-data/full_data.csv")

#### What does the data look like?¶

In [ ]:
full_data.head()

In [ ]:
full_data.tail()

In [ ]:
print("External Data")
print(f"Earliest Entry: {full_data['date'].min()}")
print(f"Last Entry:     {full_data['date'].max()}")
print(f"Total Days:     {pd.to_datetime(full_data['date']).max() - pd.to_datetime(full_data['date']).min()}")

In [ ]:
print("rows and columns")
full_data.shape

<br>

In [ ]:
#data manipulations

# only use population information of latest year and rename columns
population_data_current = population_data.loc[:,["Country Name", "2018"]]
population_data_current = population_data_current.rename(columns={'Country Name': 'location', '2018': 'population_2018'})

#merge dataset with population information
full_data = full_data.merge(population_data_current, how="left", on="location")

#get current date and yesterday's date since data does not always represents today's date.
d = datetime.today()
current_date = datetime.strftime(datetime.now() - timedelta(0), '%Y-%m-%d')
yesterdays_date = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')

#get dates and days
full_data["days"] = pd.to_timedelta(pd.to_datetime(full_data["date"]) - pd.to_datetime("2019-12-31")).dt.days
full_data["date"] = full_data["date"].astype('O')

#create relative numbers for the countries
full_data["new_cases/1M_population"] = full_data["new_cases"] / full_data["population_2018"] * 1000000
full_data["new_deaths/1M_population"] = full_data["new_deaths"] / full_data["population_2018"] * 1000000
full_data["total_cases/1M_population"] = full_data["total_cases"] / full_data["population_2018"] * 1000000
full_data["total_deaths/1M_population"] = full_data["total_deaths"] / full_data["population_2018"] * 1000000

#define interesting countries
interesting_countries_list = ["United States", "China", "Italy", "Germany", "France", "Spain", "South Korea", "Switzerland"]

#create new dataframes based for plotting.
#get only current date
full_data_today = full_data[full_data["date"] == full_data["date"].max()]
full_data_today_sorted = full_data_today.sort_values("total_cases", ascending = False).set_index(['date'])
interesting_countries_data_today = full_data_today_sorted[full_data_today_sorted["location"].isin(interesting_countries_list)]
interesting_countries_data_today = interesting_countries_data_today.sort_values("new_cases", ascending = False)
interesting_countries_data_today_new_cases = interesting_countries_data_today.groupby("location")["new_cases"].sum()

#create new dataframes based for plotting. Referring to interesting countries and cases over 200
full_data_interesting_countries = full_data.loc[(full_data["location"].isin(interesting_countries_list)) & (full_data["total_cases"] > 200)]
full_data_interesting_countries_long = full_data.loc[(full_data["location"].isin(interesting_countries_list))]

#prepare comparison from ground0 for each country

country_dict = {}
for country in full_data_interesting_countries["location"].unique():
    value_length = full_data_interesting_countries[full_data_interesting_countries["location"] == country]["location"].count()
    country_dict.update({country : value_length})

ranges= []
for key in country_dict:
    for i in range(0,country_dict[key]):
        ranges.append(int(i))

full_data_interesting_countries["day0"] = ranges

minkey = min(country_dict, key=country_dict.get)
minvalue = country_dict[minkey]

full_data_interesting_countries2 = full_data_interesting_countries[full_data_interesting_countries["day0"] < minvalue]

#preparing data to plot by category, not by country
interesting_countries_df_date = full_data_interesting_countries_long.set_index("date")
interesting_countries_df = full_data_interesting_countries_long

<br>

## 2. Initial Situation in Korea
<br>

#### South Korea was the second country (after China) to have the coronavirus outbreak, I would like to look at how they dealt with it and how they compare to European countries.
#### On Feb 2020 President Moon is under a lot of pressure because he gave masks and medical equipment to China for free and these components are now missing in Korea...his decision to hire the brightest minds in the country as advisors and follow their proposals proves to be a stroke of luck
#### The Korean outbreak has not been recognized right away. The country had a church-like cult community that could potentially spread the virus in the entire country. Several days have already passed and the government had to act quickly. Weeks after, we know that their efforts completely stopped the outbreak and the pandemic is as much under control as possible. 
#### There were no travel bans or long-lasting curfews put in place. Isntead it was data driven methods that slowed down infection rates.
#### Korea does not face a second wave of infections as most European countries do. We can learn from their efforts and prepare as well as possible for our curfews to run out.

In [ ]:
Image("../input/coronavirus-images-data/korean_politicians.jpg")

###### South Korea's President Moon Jae-in salutes to the national flag before a meeting with officials in Daegu, South Korea, February 25, 2020. Yonhap via Reuters
<br>

## 3. Exploratory Data Analysis
<br>

### How are the countries currently doing?

In [ ]:
fig = plt.figure(figsize=(15, 12))
ax = fig.gca()

plt.style.use(['dark_background'])

interesting_countries_data_today_total_cases = interesting_countries_data_today.groupby("location")["total_cases"].sum()
interesting_countries_data_today_total_cases.sort_values(ascending=True).plot(kind='barh', color=('Darkcyan'), alpha=0.8)

plt.title('Total cases by country', size=20, family = "Helvetica", weight= "bold")
plt.ylabel('Country Name', family = "Helvetica", size=15)
plt.xlabel(current_date, family = "Helvetica", size=13)
plt.xticks(rotation=45, family = "Helvetica", size=10)
plt.yticks(family = "Helvetica", size=14)

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(3.5)
    
#plt.grid(b=None)


plt.show()

#### The highest number of cases currently occured in the US.
<br>
<br>

In [ ]:
fig = plt.figure(figsize=(14, 10))
ax = fig.gca()

interesting_countries_data_today_total_deaths = interesting_countries_data_today.groupby("location")["total_deaths"].sum()
interesting_countries_data_today_total_deaths.sort_values(ascending=True).plot(kind='barh', color=('Moccasin'), alpha=0.85)


plt.title('Total deaths by country', size=20, family = "Helvetica", weight= "bold")
plt.ylabel('Country Name', family = "Helvetica", size=15)
plt.xlabel(current_date, family = "Helvetica", size=13)
plt.xticks(rotation=45, family = "Helvetica", size=10)
plt.yticks(family = "Helvetica", size=14)


plt.style.use(['dark_background'])
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(3.5)
#plt.grid(b=None)

plt.show()

#### The highest number of deaths has occured in Italy with more than 10'000.
<br>
<br>

In [ ]:
fig = plt.figure(figsize=(15, 12))
ax = fig.gca()

plt.style.use(['dark_background'])

interesting_countries_data_today_total_cases2 = interesting_countries_data_today.groupby("location")["total_cases/1M_population"].sum()
interesting_countries_data_today_total_cases2.sort_values(ascending=True).plot(kind='barh', color=('Cyan'), alpha=0.8)

plt.title('Total cases by country and per 1 million residents', size=20, family = "Helvetica", weight= "bold")
plt.ylabel('Country Name', family = "Helvetica", size=15)
plt.xlabel(current_date, family = "Helvetica", size=13)
plt.xticks(rotation=45, family = "Helvetica", size=10)
plt.yticks(family = "Helvetica", size=14)

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(3.5)
    
#plt.grid(b=None)


plt.show()

#### The most cases relative to the number of residents occur in Spain, followed by Switzerland and Italy.
<br>
<br>

### Situation by country

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.gca()
#plt.plot(full_data_interesting_countries.groupby(["days"]).sum()["total_cases"] )
plt.plot(full_data_interesting_countries.groupby(['days','location']).sum()['total_cases'].unstack())
#plt.plot(interesting_countries_df.groupby(['date','location']).sum()['total_cases'].unstack())
#full_data.loc[full_data["location"] == country].plot()

plt.title("total_cases by country", size=32)
plt.legend(labels =full_data_interesting_countries["location"].unique(), ncol=2, fontsize="large")
plt.xlabel("days since 2019/12/31" ,size=24)
plt.ylabel("# of cases", size=24)
#plt.grid(b=None)
#plt.gca(visible=False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()

#### Only China and Korea could significantly slow down the spread. The European efforts still are pending to be effective as total cases are still rising

#### As you can see China encountered the first cases, followed by South Korea and then Italy. Very noticeably is the shorter timeframe from country to country. This is to be expected in other world regions due to the close proximity and people passing from a country to another.¶

#### Switzerland shows a lot lower cases than for example Germany. The case per inhabitant in Switzerland is almost as high as in Italy and that is although the Italian outbreak happened 10 days earlier than the Swiss and although testing capabilities in Switzerland are limited.

#### Koreans suffered from the virus about 5-10 days earlier than the European countries. The outbreak, however, seems to be under control now. 
<br>
<br>

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.gca()
#plt.plot(full_data_interesting_countries.groupby(["days"]).sum()["total_cases"] )
plt.plot(full_data_interesting_countries.groupby(['days','location']).sum()['new_cases'].unstack())
#plt.plot(interesting_countries_df.groupby(['date','location']).sum()['total_cases'].unstack())
#full_data.loc[full_data["location"] == country].plot()

plt.title("new_cases by country", size=32)
plt.legend(labels =full_data_interesting_countries["location"].unique(), ncol=2, fontsize="large")
plt.xlabel("days since 2019/12/31" ,size=24)
plt.ylabel("# of cases", size=24)
#plt.grid(b=None)
#plt.gca(visible=False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()

#### New cases happen mainly in Europe and the US
#### Clearly, the European efforts show some effect.
<br>
<br>

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.gca()
#plt.plot(full_data_interesting_countries.groupby(["days"]).sum()["total_cases"] )
plt.plot(full_data_interesting_countries.groupby(['days','location']).sum()['total_deaths'].unstack())
#plt.plot(interesting_countries_df.groupby(['date','location']).sum()['total_cases'].unstack())
#full_data.loc[full_data["location"] == country].plot()

plt.title("total_deaths by country", size=32)
plt.legend(labels =full_data_interesting_countries["location"].unique(), ncol=2, fontsize="large")
plt.xlabel("days since 2019/12/31" ,size=24)
plt.ylabel("# of cases", size=24)
#plt.grid(b=None)
#plt.gca(visible=False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()

#### The total deaths are rising. Several countries do not contribute much to the total deaths yet, including Germany, South Korea and Switzerland
#### China does not encounter more deaths due to the virus. It looks like it has stabilized completely.
#### The US will soon be passing Italy and Spain in total number of deaths.
#### Korea does not show the rise in total deaths that we see in the European countries yet. This might be due to the low number of overall infected in Korea.
<br>
<br>

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.gca()
#plt.plot(full_data_interesting_countries.groupby(["days"]).sum()["total_cases"] )
plt.plot(full_data_interesting_countries.groupby(['days','location']).sum()['new_deaths'].unstack())
#plt.plot(interesting_countries_df.groupby(['date','location']).sum()['total_cases'].unstack())
#full_data.loc[full_data["location"] == country].plot()

plt.title("new_deaths by country", size=32)
plt.legend(labels =full_data_interesting_countries["location"].unique(), ncol=2, fontsize="large")
plt.xlabel("days since 2019/12/31" ,size=24)
plt.ylabel("# of cases", size=24)
#plt.grid(b=None)
#plt.gca(visible=False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()

#### The new deaths will rise until the new cases are under control. There has been no efficient medication on the market against Coronavirus. The European countries did not manage to reduce the number of deaths.
<br>
<br>

### Let's put the countries into perspective with each other. If all countries started with the virus during the same date, how would the first 3 weeks look like?

In [ ]:
xint = range(0, minvalue)

fig = plt.figure(figsize=(16, 9))
ax = fig.gca()
#plt.plot(full_data_interesting_countries.groupby(["days"]).sum()["total_cases"] )
plt.plot(full_data_interesting_countries2.groupby(['day0','location']).sum()['total_cases'].unstack())
#plt.plot(interesting_countries_df.groupby(['date','location']).sum()['total_cases'].unstack())
#full_data.loc[full_data["location"] == country].plot()

plt.title("total_cases by country", size=32)
plt.legend(labels =full_data_interesting_countries["location"].unique(), ncol=2, fontsize="x-large")
plt.xlabel("days since # of cases in country reached more than total of 200" ,size=24)
plt.ylabel("# of cases", size=24)
plt.xticks(xint)
#plt.grid(b=None)
#plt.gca(visible=False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()

#### The US show the strongest rise in positive cases. This might be due to a long time of unawareness or to a rise of testing possibilities for the residents. It might also be a combination of both.
#### Following are Italy and China
#### Korea has the lowest overall case number and that is although Switzerland has only about 16% of the Korean population!
#### During this time period Korea has performed about 300'000 tests. Only about 8'000 were positive. Switzerland performed about 50'000 tests, that means that the true number of positive cases is expected to be even higher.
###### sources: https://nzzas.nzz.ch/wissen/coronavirus-ueber-50000-tests-in-der-schweiz-ld.1547810?reduced=true ; https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030
#### Another important fact to understand is that before a positve test result is recognized about 8-12 days pass. After infection, people take between 1 day and 14 days to show symptoms and then get tested. Test results take a few hours (depending on the county) before the result is shown.
<br>
<br>

### Let's compare the same numbers but in relation to the total number of residents in a given country.

In [ ]:
xint = range(0, minvalue)

fig = plt.figure(figsize=(16, 9))
ax = fig.gca()
#plt.plot(full_data_interesting_countries.groupby(["days"]).sum()["total_cases"] )
plt.plot(full_data_interesting_countries2.groupby(['day0','location']).sum()['total_cases/1M_population'].unstack())
#plt.plot(interesting_countries_df.groupby(['date','location']).sum()['total_cases'].unstack())
#full_data.loc[full_data["location"] == country].plot()

plt.title("total_cases by country per 1M population", size=32)
plt.legend(labels =full_data_interesting_countries["location"].unique(), ncol=2, fontsize="x-large")
plt.xlabel("days since # of cases in country reached more than total of 200" ,size=24)
plt.ylabel("# of cases", size=24)
plt.xticks(xint)
#plt.grid(b=None)
#plt.gca(visible=False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()

#### It does look very different. Switzerland who has been among the lowest total numbers shows the highest increase per resident within the first 21 days.
<br>
<br>

### How does it look like comparing only the last month and per residents in a given country?

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.gca()
#plt.plot(full_data_interesting_countries.groupby(["days"]).sum()["total_cases"] )
plt.plot(full_data_interesting_countries[full_data_interesting_countries["days"]> 55].groupby(['days','location']).sum()['total_cases/1M_population'].unstack())
#plt.plot(interesting_countries_df.groupby(['date','location']).sum()['total_cases'].unstack())
#full_data.loc[full_data["location"] == country].plot()

plt.title("total_cases by country per 1M_population", size=32)
plt.legend(labels =full_data_interesting_countries["location"].unique(), ncol=2, fontsize="large")
plt.xlabel("days since 2019/12/31" ,size=24)
plt.ylabel("# of cases per 1M_population", size=24)
#plt.grid(b=None)
#plt.gca(visible=False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()

#### Spain, Switzerland and Italy have the highest infection rate within their populations.
<br>
<br>

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.gca()
#plt.plot(full_data_interesting_countries.groupby(["days"]).sum()["total_cases"] )
plt.plot(full_data_interesting_countries[full_data_interesting_countries["days"]> 55].groupby(['days','location']).sum()['total_deaths/1M_population'].unstack())
#plt.plot(interesting_countries_df.groupby(['date','location']).sum()['total_cases'].unstack())
#full_data.loc[full_data["location"] == country].plot()

plt.title("total_deaths by country per 1M_population", size=32)
plt.legend(labels =full_data_interesting_countries["location"].unique(), ncol=2, fontsize="large")
plt.xlabel("days since 2019/12/31" ,size=24)
plt.ylabel("# of cases per 1M_population", size=24)
#plt.grid(b=None)
#plt.gca(visible=False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()

#### In terms of total deaths per 1 million residents, Switzerland is still a lot lower than Italy and Spain. Considering that the spread started later in Switzerland, we could fear that the total number of deaths could rise too.
<br>
<br>

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.gca()
#plt.plot(full_data_interesting_countries.groupby(["days"]).sum()["total_cases"] )
plt.plot(full_data_interesting_countries[full_data_interesting_countries["days"]> 55].groupby(['days','location']).sum()['new_deaths/1M_population'].unstack())
#plt.plot(interesting_countries_df.groupby(['date','location']).sum()['total_cases'].unstack())
#full_data.loc[full_data["location"] == country].plot()

plt.title("new_deaths by country per 1M_population", size=32)
plt.legend(labels =full_data_interesting_countries["location"].unique(), ncol=2, fontsize="large")
plt.xlabel("days since 2019/12/31" ,size=24)
plt.ylabel("# of cases per 1M_population", size=24)
#plt.grid(b=None)
#plt.gca(visible=False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()

#### Looking at the new_death rate per 1 million population, it looks like Switzerland is more similar to France.

#### The total deaths are still rising. In fact, the European countries are bound to decide now which patient will get an intensive care bed and which one cannot. Low number of beds per inhabitant and the ineffective way of dealing with the crisis are the main reasons for it. 
<br>
<br>
<br>

## 4. Discussion & Recommendation
<br>

### South Korea does not have more beds than others.

In [ ]:
Image("../input/coronavirus-images-data/care beds.jpg")

###### source: https://www.forbes.com/sites/niallmccarthy/2020/03/12/the-countries-with-the-most-critical-care-beds-per-capita-infographic/#9eb8ba47f864

<br>

### So what did South Korea exactly do to reduce the spread and deaths?
<br>

### 1. Testing - as much, early and safe as possible.
#### The Drive Throughs were rolled out country wide very quickly.

In [ ]:
Image("../input/coronavirus-images-data/Korean Drive Through.jpg")

###### source: https://newsinfo.inquirer.net/1247149/experience-from-mers-made-koreas-covid-19-test-kits-possible
<br>

### 2. Information & Detection Systems.
#### Testing was not simply performed but positive patients received the information via SMS within hours and got directed to a link where they could enter information on their whereabouts and contacts during the last days. The government could then shut down specific areas and inform local residents to be careful. The annoying SMS sounds could not be stopped and people listened to it.
<br>

### 3. Desinfection and Masks
#### The state closed down business and even whole train stations were infected passed time. They infected the entire area, shut it down for 12 hours and reopened after!
#### Every politician is wearing a mask. It is part of good manners to protect the elderly and not questionable.

In [ ]:
Image("../input/coronavirus-images-data/dongdaegu yeok.jpg")

###### source: https://www.nbcnews.com/news/world/coronavirus-updates-10-italian-communities-lockdown-death-toll-rises-china-n1141321
<br>

### 4. Transparency
#### The official government website CDC published current numbers up to three times per day. People were also informed about the spread in different regions, cities and current measures efficiently. The Korean government tried to find out the infection chain of each positive case. Countless people got involved voluntarily in order to find out.¶
#### Additionally there are a number of available apps that show anonymized data as detailed as 100m.

In [ ]:
Image("../input/coronavirus-images-data/Corona App.jpg")

###### source: https://edition.cnn.com/2020/02/28/tech/korea-coronavirus-tracking-apps/index.html
<br>

### 5. Culture
#### Voluntary work was a well-received and good task to perform in times of crisis. People packed bags for infected or workers in need, helped each other on social media and took care of elderly.

In [ ]:
Image("../input/coronavirus-images-data/voluntary lunch.jpg")

###### source: http://www.koreaherald.com/view.php?ud=20200322000108
<br>

### 6. Follow up on actions taken
#### Each and every measure has been data driven. Numbers were thoroughly looked at when deciding and this is sill the case.
<br>

## Does this help us?
### YES, absolutely! We can and should implement the systems & measures that helped Korea.
#### In the EU many argue that data protection hinders us from following what Korea does. It does not. Anonymized data can be well shown and also analyzed. The Robert Koch Institut in Germany for example looks at mobility data to see if lockdown measures work as expected. Data protection does certainly slow our efforts down due to misunderstandings but it does not block us in any other way. 
###### source: https://www.welt.de/vermischtes/article206812919/Auswirkungen-der-Corona-Massnahmen-So-drastisch-nimmt-der-Verkehr-ab.html
#### We have a very rigid lockdown at the moment, is that the right thing to do? Maybe butw e must expect that there is a second wave of infections coming once we release the shutdown rules. It does however protect us from an initial strong hit that medical facilities cannot cope with. And most importantly it gives us time now to prepare for said second wave of infections.
#### Korea did not stop foreigners from entering the country even thuogh 1 Million Chinese students came in March 2020. Once the systems are in place, free mobility is an option again!
<br>

### How does the outlook look like? Is the crisis almost over or just the beginning?


In [ ]:
temp = full_data.groupby(['date', 'location'])['total_cases'].sum().reset_index()
temp['date'] = pd.to_datetime(temp['date'])
temp['date'] = temp['date'].dt.strftime('%m/%d/%Y')
temp['size'] = temp['total_cases'].pow(0.3) * 3.5

fig = px.scatter_geo(temp, locations="location", locationmode='country names', 
                     color="total_cases", size='size', hover_name="location", 
                     range_color=[1,100],
                     projection="natural earth", animation_frame="date", 
                     title='COVID-19: Cases Over Time', color_continuous_scale="greens")
fig.show()

#### The crisis just began. Countries are to be expected to be infected although drastic measures have been taken. If other countries follow the Korean example we can at least slow down the development and save thousands of lives.
<br>
<br>
<br>
<br>